In [9]:
import warnings
warnings.filterwarnings('ignore')

from pprint import pprint
from time import time
import logging

import pandas as pd

import numpy as np
import sklearn as sk
import seaborn as sns

import nltk
from nltk import word_tokenize
lang ='french'

import clean_text
import skmultilearn




import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score,classification_report
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import TruncatedSVD,IncrementalPCA,SparsePCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier

import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

# Annalyse des signalement de la DGS grace aux techniques de l'inteligence artificielle

## Description

### Le projet
Le projet se déroule sur 8 mois d'Avril à Novembre 2020 et il se compose de 4 livrables distincts : 
- Livrable 1 : SOCLE TECHNIQUE D’INGESTION DES DONNEES
- Livrable 2 : ALGORITHMES DE CLASSIFICATION DES SIGNALEMENTS
- Livrable 3 : ALGORITHME DE REGROUPEMENT DES SIGNALEMENTS
- Livrable 4 : INTERFACE DE TRAITEMENTS DES SIGNALEMENTS

Une description plus complète de chacun des livrables se trouve dans la réponse à l'appel d'offre.

### Organisation du GitLab
Ce répetroire git est organisé en 3 branches différentes : 
- **Exploration :** 
Cette branche est essentiellement composée de notebook retraçant les différentes pistes que nous avons suivies pour arriver à la solution finale. Cette branche permet de rendre compte de la démarche suivie et ainsi de justifier de la qualité scientifique du livrable.  A travers ces notebooks d'exploration, nous pourrons également envisager les directions futures pour améliorer les performances

- **Demonstrateur :**
Cette branche contient un démonstrateur permettant de visualiser les résultats des modèles. Cette branche est un livrable intermédiaire qui nous permet de dialoguer avec les personnes de la DGS et ainsi de co-construire les livrables 2, 3 et 4 en mettant en évidence les besoins de la DGS.

- **Develop ,** Cette branche contient les livrables dans différents dossiers : 
    - __extractor__: le script qui permet d'ingérer les données PDF (Livrable 1)
    - __inference__: Le script et les modèles d'inférence de la DCO, de la TYPOLOGIE et de la GRAVITÉ pour une base de fichier PDF ou XML (Livrable 2)
    - __regroupement__ : Les algorithmes et les modèles pour le regroupement des signalements (Livrable 3)
    - __application__  : Le code source pour l'interface de traitement (Livrable 4)

## Pour Commencer(a faire)



## Authors

* **Robin Quillivic**  
* **Louise Remot**  
* **Boris Sanchot**  

## License

Ce projet est privé.
This project is completely private.


In [12]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [109]:
mrv = pd.read_csv('data/data_mrv/declaration_mrv_complet.csv')

In [110]:
mrv.columns

Index(['Unnamed: 0', 'NUMERO_DECLARATION', 'TYPE_DECLARATION',
       'TYPE_DECLARANT', 'NB_PATIENT_CONCERNE', 'NB_DISPOSITIF_CONCERNE',
       'DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'ACTION_PATIENT', 'DCO_ID',
       'DCO', 'LIBELLE_COMMERCIAL', 'REFERENCE_COMMERCIALE', 'NUMERO_SERIE',
       'NUMERO_LOT', 'FABRICANT', 'MANDATAIRE', 'DISTRIBUTEUR', 'TDY_ID',
       'TYPE_DYSFONCTIONNEMENT', 'CDY_ID', 'CONSEQUENCE_DYSFONCTIONNEMENT',
       'TEF_ID', 'TYPE_EFFET', 'GRAVITE', 'NUMERO', 'TYPE_VIGILANCE',
       'CLASSIFICATION'],
      dtype='object')

In [125]:
len(mrv[mrv.MANDATAIRE.isna()])

62277

In [192]:
import pandas as pd
import joblib
import numpy as np
import clean_text

from sklearn.preprocessing import LabelEncoder
import os

directory = './data_split/'
if not os.path.exists(directory):
    os.makedirs(directory)

# Chargement des données
mrv = pd.read_csv('data/data_mrv/declaration_mrv_complet.csv',
                 usecols=['NUMERO_DECLARATION',"DESCRIPTION_INCIDENT","ETAT_PATIENT","FABRICANT", "CLASSIFICATION", 'ACTION_PATIENT',
                          'DCO_ID','LIBELLE_COMMERCIAL','REFERENCE_COMMERCIALE','TEF_ID',
                          'TDY_ID','CDY_ID','GRAVITE'])

# 1) On gère nana pour les collones contenant du text
text_columns =  ["DESCRIPTION_INCIDENT","ETAT_PATIENT","FABRICANT", 'ACTION_PATIENT','LIBELLE_COMMERCIAL']


mrv[text_columns] = mrv[text_columns].fillna('NON RENSEIGNE')

# 2) on gère les NaN pour chaque Collones
mrv['NUMERO_DECLARATION'] = mrv['NUMERO_DECLARATION'].fillna('NON RENSEIGNE')
mrv['DESCRIPTION_INCIDENT'] = mrv['DESCRIPTION_INCIDENT'].fillna('NON RENSEIGNE')
mrv['ETAT_PATIENT'] = mrv['ETAT_PATIENT'].fillna('NON RENSEIGNE')              
mrv['CLASSIFICATION'] = mrv['CLASSIFICATION'].fillna('NON RENSEIGNE')
mrv['FABRICANT'] = mrv['FABRICANT'].fillna('NON RENSEIGNE')
mrv['LIBELLE_COMMERCIAL'] = mrv['LIBELLE_COMMERCIAL'].fillna('NON RENSEIGNE')
mrv['REFERENCE_COMMERCIALE'] = mrv['REFERENCE_COMMERCIALE'].fillna('NON RENSEIGNE')



#Effet
mrv['TEF_ID']= mrv['TEF_ID'].fillna('E1213')
#DYSFOCNTIONNEMENT
mrv['TDY_ID']  = mrv['TDY_ID'].fillna("D0")
#CONSEQUENCES
mrv['CDY_ID']  = mrv['CDY_ID'].fillna("C0")

#Construction de nouvelles variables

mrv["text"] = mrv['DESCRIPTION_INCIDENT']+' '+mrv['ETAT_PATIENT']


# Encodage des variables catégorielles et sauvegarde des données associé

le = LabelEncoder()
mrv["CLASSIFICATION"] = le.fit_transform(mrv["CLASSIFICATION"])
joblib.dump(le, './data_split/classification_encodeur.sav')


le = LabelEncoder()
mrv["CDY_ID"] = le.fit_transform(mrv["CDY_ID"])
joblib.dump(le, './data_split/CDY_ID_encodeur.sav')


le = LabelEncoder()
mrv["TEF_ID"] = le.fit_transform(mrv["TEF_ID"])
joblib.dump(le, './data_split/TEF_ID_encodeur.sav')


le = LabelEncoder()
mrv["TDY_ID"] = le.fit_transform(mrv["TDY_ID"])
joblib.dump(le, './data_split/TDY_ID_encodeur.sav')



# La typologie est en génrale multilabel

mrv_id = mrv.groupby('text').agg({'TEF_ID':lambda x: list(set(x)),
                               'CDY_ID':lambda x: list(set(x)),
                               'TDY_ID':lambda x:list(set(x))
                            })

#On fusionne les multilabels et on supprime les doublons
mrv = mrv.drop(['TEF_ID','CDY_ID','TDY_ID'],axis=1)
mrv = pd.merge(mrv,mrv_id, on = 'text')
mrv = mrv.drop_duplicates('text')
mrv = mrv.reset_index(drop=True)


#On remplace les strings vide
mrv.replace('', 'NON RENSEIGNE', inplace=True)
# Sauvegarde des données nettoyées
mrv.to_pickle('./data_split/all_data.pkl')


# Split data

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1029)

mlb = MultiLabelBinarizer()

for train_index, test_index in msss.split(mrv['text'], mlb.fit_transform(mrv['TEF_ID'])):
    train, test = mrv.loc[train_index],  mrv.loc[test_index]

train.to_pickle('./data_split/train.pkl')
test.to_pickle('./data_split/test.pkl')

print(train.shape, test.shape)

(26324, 14) (6580, 14)


In [182]:
mrv_id[['TEF_ID','CDY_ID','TDY_ID']]=mrv_id[['TEF_ID','CDY_ID','TDY_ID']].apply(lambda x :np.array(x))

In [183]:
mlb = MultiLabelBinarizer()
a = mlb.fit_transform(mrv_id['TEF_ID'])
a.shape

(32731, 273)

In [160]:
mrv.isna().sum()

DESCRIPTION_INCIDENT        0
ETAT_PATIENT                0
ACTION_PATIENT              0
DCO_ID                      6
LIBELLE_COMMERCIAL          0
REFERENCE_COMMERCIALE       0
FABRICANT                   0
GRAVITE                  1290
CLASSIFICATION              0
text                        0
TEF_ID                      0
CDY_ID                      0
TDY_ID                      0
dtype: int64

In [173]:
len(mrv[mrv['ETAT_PATIENT']==''])

453

In [185]:
mrv.to_pickle('test.pkl')
data = pd.read_pickle('test.pkl')
data.isna().sum()

DESCRIPTION_INCIDENT        0
ETAT_PATIENT                0
ACTION_PATIENT              0
DCO_ID                      6
LIBELLE_COMMERCIAL          0
REFERENCE_COMMERCIALE       0
FABRICANT                   0
GRAVITE                  1290
CLASSIFICATION              0
text                        0
TEF_ID                      0
CDY_ID                      0
TDY_ID                      0
dtype: int64

In [140]:
train.head()

,DESCRIPTION_INCIDENT,ETAT_PATIENT,ACTION_PATIENT,DCO_ID,LIBELLE_COMMERCIAL,REFERENCE_COMMERCIALE,FABRICANT,TYPE_DYSFONCTIONNEMENT,CONSEQUENCE_DYSFONCTIONNEMENT,TYPE_EFFET,GRAVITE,CLASSIFICATION,text,TEF_ID,CDY_ID,TDY_ID
1,apparition de nombreux boutons,,,3268.0,tensoban - bande de protection adhesive,..,bsn medical,NON RENSEIGNE,PAS DE CONSEQUENCE,REACTION ALLERGIQUE,MOYEN,1,apparition de nombreux boutons,[123],[0],[337]
2,"le sphinctérotome , à la sortie de son emballa...",. .,,2497.0,sphincterotome dreamtome rx,M00584050,boston scientific,NON EFFICACITE DM,PAS DE CONSEQUENCE,PAS D'EFFET NEFASTE DECLARE,NULLE,1,"le sphinctérotome , à la sortie de son emballa...",[106],[0],[329]
3,mandrin bloqué dans la sonde et impossibilité ...,. .,matériel à dispo au sce économat .,2851.0,quartet,1456Q86,abbott medical france sas,IMPOSSIBILITE / DIFFICULTE DE POSE / MISE EN P...,PAS DE CONSEQUENCE,PAS D'EFFET NEFASTE DECLARE,MOYEN,1,mandrin bloqué dans la sonde et impossibilité ...,[106],[0],[286]
4,la pince kocher présente dans la trousse a été...,"perte sanguine du bébé , la pince ne réalisant...",un clamp ombilical a été mis en urgence .,4841.0,pince kocher dans la trousse accouchement,0007255334,vygon,PROBLEME OUVERTURE / FERMETURE DE LA PINCE,PAS DE CONSEQUENCE,PERTE DE SANG,MOYEN,1,la pince kocher présente dans la trousse a été...,[113],[0],[521]
5,lors de ouverture un pack cataracte ( personna...,pas de conséquences car non utilisation du dm .,changements de couteaux .,2902.0,couteau standart ° droit st,..,bausch & lomb france,DM INCOMPLET / DETERIORE AVANT UTILISATION,PAS DE CONSEQUENCE,PAS D'EFFET NEFASTE DECLARE,MINEU,1,lors de ouverture un pack cataracte ( personna...,[106],[0],"[395, 206]"


In [142]:
data = pd.read_csv('data/data_mrv/declaration_mrv_complet.csv',
                 usecols=["DESCRIPTION_INCIDENT","ETAT_PATIENT","FABRICANT", "CLASSIFICATION", 'ACTION_PATIENT',
                          'DCO_ID','LIBELLE_COMMERCIAL','REFERENCE_COMMERCIALE','TYPE_EFFET','TEF_ID','TYPE_DYSFONCTIONNEMENT',
                          'TDY_ID','CONSEQUENCE_DYSFONCTIONNEMENT','CDY_ID','GRAVITE'])

In [145]:
data["text"] = data['DESCRIPTION_INCIDENT']+' '+data['ETAT_PATIENT']
data_id = data.groupby('text')
len(data_id)

32452

In [156]:
len(data['DCO_ID'].unique())

1094

In [80]:
X_train = train[["DESCRIPTION_INCIDENT","ETAT_PATIENT","FABRICANT","CLASSIFICATION"]]
X_test = test[["DESCRIPTION_INCIDENT","ETAT_PATIENT","FABRICANT","CLASSIFICATION"]]
y_train = mlb.transform(train['TEF_ID'])
y_test = mlb.transform(test['TEF_ID'])
print(y_train.shape, y_test.shape, X_train.shape,X_test.shape)

(8649, 266) (2099, 266) (8649, 4) (2099, 4)


In [ ]:
(11627, 269) (2896, 269) (11627, 4) (2896, 4)

In [108]:
11627+2896 - len(mrv)

0

In [105]:
X_train['len'] = X_train['FABRICANT'].map(lambda x : len(x))
X_train[X_train['len']>1]['len'].sum()

1752

In [71]:
%%time
preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT'),
    
    ('classification_enc', TfidfVectorizer(sublinear_tf=True, min_df=5,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 100,norm = 'l2'),'CLASSIFICATION')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 11.5 ms


In [76]:
%%time
pipeline.fit(X_train,y_train)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

AttributeError: 'list' object has no attribute 'lower'

In [73]:
print(classification_report(y_test , Y_pred_ovr))

              precision    recall  f1-score   support

           0       0.75      0.43      0.55         7
           1       1.00      0.25      0.40         4
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.43      0.50      0.46         6
           5       0.00      0.00      0.00         1
           6       0.50      0.12      0.20         8
           7       0.62      0.45      0.53        33
           8       0.25      0.19      0.22        36
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         2
          11       0.33      0.14      0.20         7
          12       0.33      0.06      0.10        18
          13       0.00      0.00      0.00         5
          14       1.00      0.33      0.50         9
          15       1.00      0.12      0.22         8
          16       1.00      0.14      0.25         7
          17       1.00    

In [61]:
%%time
pipeline.fit(X_train,y_train)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.5264524271227642
CPU times: user 17.9 s, sys: 116 ms, total: 18 s
Wall time: 18 s


In [62]:
print(classification_report(y_test , Y_pred_ovr))

              precision    recall  f1-score   support

           0       1.00      0.29      0.44         7
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.69      0.64      0.67        14
           5       0.00      0.00      0.00         1
           6       0.25      0.10      0.14        10
           7       0.56      0.55      0.55        44
           8       0.41      0.23      0.29        48
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         2
          11       0.67      0.25      0.36         8
          12       0.30      0.15      0.20        20
          13       1.00      0.40      0.57         5
          14       1.00      0.11      0.20         9
          15       0.33      0.11      0.17         9
          16       0.00      0.00      0.00         7
          17       0.67    

In [40]:
%%time
pipeline.fit(X_train,y_train)
Y_pred_ovr = pipeline.predict(X_test)
f1 = f1_score(y_test , Y_pred_ovr,average='samples')
print('f1_score samples : ',f1)

f1_score samples :  0.9016472532225805
CPU times: user 2min 8s, sys: 284 ms, total: 2min 9s
Wall time: 2min 9s


In [45]:
print(Y_pred_ovr.shape, y_test.shape)

(9014, 269) (9014, 269)


In [41]:
print(classification_report(y_test , Y_pred_ovr))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        39
           1       1.00      0.29      0.44         7
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00        19
           4       0.92      0.97      0.94        68
           5       1.00      1.00      1.00         4
           6       0.95      0.91      0.93        22
           7       0.91      0.91      0.91        89
           8       0.97      0.99      0.98       365
           9       1.00      0.25      0.40         4
          10       1.00      1.00      1.00        16
          11       1.00      0.87      0.93        30
          12       0.86      0.82      0.84        45
          13       1.00      0.82      0.90        11
          14       1.00      0.93      0.97        30
          15       1.00      0.83      0.90        23
          16       0.89      0.67      0.76        12
          17       1.00    

In [58]:
y_train.sum()

11628

In [193]:
%time

mlb = MultiLabelBinarizer()

train = pd.read_pickle('./data_split/train.pkl')
# Pour faire un modèle sans le 
#train = train[~train['TEF_ID'].map(lambda x : 106 in x)]
X_train = train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_train = mlb.fit_transform(train['TEF_ID'])
test =  pd.read_pickle('./data_split/test.pkl')
#test = test[~test['TEF_ID'].map(lambda x : k in x)]
X_test = test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_test = mlb.transform(test['TEF_ID'])


df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.29 ms


In [210]:
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier

preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT')
     ],
    
    remainder='passthrough')

preprocess_2 = ColumnTransformer(
    [('description_tfidf',CountVectorizer( min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', CountVectorizer( min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',CountVectorizer(min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000), 'FABRICANT')
     ],
    
    remainder='passthrough')


pipeline = Pipeline([
    ('vect', preprocess),
    ('svd', TruncatedSVD(n_components=1000)),
    ('clf', OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, class_weight="balanced")))
])

pipeline_2 = Pipeline([
    ('vect', preprocess_2),
    ('svd', TruncatedSVD(n_components=1000)),
    ('clf', OneVsRestClassifier(RandomForestClassifier(n_jobs=-1)))
])

In [ ]:
RandomForestClassifier()

In [211]:
%%time
pipeline.fit(X_train,y_train)
y_pred =pipeline.predict(X_test)
print(f1_score(y_test,y_pred, average='samples'))

0.578100646675142
CPU times: user 7min 49s, sys: 3min 5s, total: 10min 55s
Wall time: 30min 58s


In [209]:
%%time
pipeline_2.fit(X_train,y_train)
y_pred =pipeline_2.predict(X_test)
print(f1_score(y_test,y_pred, average='samples'))

0.5260033309590517
CPU times: user 7.78 s, sys: 88 ms, total: 7.86 s
Wall time: 7.88 s


In [212]:
import warnings
warnings.filterwarnings('ignore')

import joblib
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,f1_score,classification_report,recall_score,precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

In [213]:
%%time

mlb = MultiLabelBinarizer()

train = pd.read_pickle('./data_split/train.pkl')
# Pour faire un modèle sans le 
#train = train[~train['TEF_ID'].map(lambda x : 106 in x)]
X_train = train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_train = mlb.fit_transform(train['TEF_ID'])
test =  pd.read_pickle('./data_split/test.pkl')
#test = test[~test['TEF_ID'].map(lambda x : k in x)]
X_test = test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_test = mlb.transform(test['TEF_ID'])


X_train_dgs = np.load('results/dgs_camenbert_train_vec.npy')
X_test_dgs =np.load('results/dgs_camenbert_test_vec.npy')



df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 28.8 µs


In [232]:
# entrainons un modèle par classes
n=1536
dgs_desc_train, dgs_etat_train, dgs_fab_train,dgs_action_train  = X_train_dgs[:,:n],X_train_dgs[:,n:2*n], X_train_dgs[:,2*n:3*n], X_train_dgs[:,3*n:]
dgs_desc_test, dgs_etat_test, dgs_fab_test,dgs_action_test  = X_test_dgs[:,:n],X_test_dgs[:,n:2*n], X_test_dgs[:,2*n:3*n], X_test_dgs[:,3*n:]

train_list = [dgs_desc_train, dgs_etat_train, dgs_fab_train,dgs_action_train]
test_list = [dgs_desc_test, dgs_etat_test, dgs_fab_test,dgs_action_test ]

pipeline = Pipeline([
    ('svd',TruncatedSVD(n_components=300)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])
PRED= []
for x_train,x_test in zip(train_list,test_list) :
    pipeline.fit(x_train,y_train)
    y_pred= pipeline.predict(x_test)
    print(f1_score(y_test, y_pred, average='samples'))
    PRED.append(y_pred)

KeyboardInterrupt: 

In [223]:
dgs_desc.shape

(26324, 1536)

In [225]:
pipeline_dgs = joblib.load('Effet_model_dgs_emb.sav')
y_pred_dgs = pipeline_dgs.predict(X_test_dgs)

In [230]:
%%time
from sklearn.decomposition import TruncatedSVD



preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT')
     ],
    
    remainder='passthrough')

pipeline = Pipeline([
    ('vect', preprocess),
    ('svd',TruncatedSVD(n_components=1000)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'))),
])

pipeline.fit(X_train,y_train)

y_pred = pipeline.predict(X_test)

print(f1_score(y_test, y_pred, average='samples'))

0.5602698959256718
CPU times: user 25min 27s, sys: 6min 18s, total: 31min 46s
Wall time: 23min 46s


In [227]:
y_pred_ensemble = np.mean([y_pred_dgs,y_pred],axis=0)

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7,0.8]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred_ensemble.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.01
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.04
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.06
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.08
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.1
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.12
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.14
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.16
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.2
Samples-average quality numbers
Precision: 0.5775, Recall: 0.7811, F1-measure: 0.6323
For threshold:  0.25
Samples-average qu